In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [3]:
IMAGE_HEIGHT, IMAGE_WIDTH = 128, 128
BATCH_SIZE = 32
base_dir = r"C:\Users\joeca\Downloads\chest_xray"

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)       
x = Dropout(0.5)(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(1, activation='sigmoid')(x)

model_1 = Model(inputs=base_model.input, outputs=output)

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1  
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_vgg16_model.h5', monitor='val_accuracy', save_best_only=True)

In [ ]:
history_1 = model_1.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


In [ ]:
test_loss, test_accuracy = model_1.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_1.history['accuracy'], label='Train Acc')
plt.plot(history_1.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_1.history['loss'], label='Train Loss')
plt.plot(history_1.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
filepath = r"C:\Users\joeca\PycharmProjects\GPU_usage\myenv\model_weights_03\vgg_model_new_01.h5"
model_1.save(filepath, overwrite=True)
print('model saved at ',filepath)

In [ ]:
val_accuracies = history_1.history['val_accuracy']
val_losses = history_1.history['val_loss']

avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)
avg_val_loss = sum(val_losses) / len(val_losses)

print(f"Average Validation Accuracy: {avg_val_accuracy * 100:.2f}%")
print(f"Average Validation Loss: {avg_val_loss:.4f}")


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

test_generator.reset()

pred_probs = model_1.predict(test_generator, verbose=1)
pred_classes = (pred_probs > 0.5).astype(int).reshape(-1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

cm = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print("\nClassification Report:")
print(classification_report(true_classes, pred_classes, target_names=class_labels))

In [ ]:
import matplotlib.pyplot as plt

# Get image file paths
filenames = test_generator.filenames
img_paths = [os.path.join(test_generator.directory, fname) for fname in filenames]

plt.figure(figsize=(15, 8))
for i in range(5):
    img = plt.imread(img_paths[i])
    label = class_labels[true_classes[i]]
    pred = class_labels[pred_classes[i]]
    plt.subplot(1, 5, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"True: {label}\nPred: {pred}", color='green' if label == pred else 'red')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt


In [ ]:
from tensorflow.keras.models import load_model

model = load_model(r"PycharmProjects/GPU_usage/myenv/model_weights_03/vgg_model_new_01.h5")
print("Model loaded successfully.")


In [ ]:
img_path = r"C:\Users\joeca\Downloads\chest_xray\test\NORMAL\NORMAL2-IM-0325-0001.jpeg"

img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0 

In [ ]:
prediction = model.predict(img_array)

class_label = "PNEUMONIA" if prediction[0][0] > 0.5 else "NORMAL"

print(f"Predicted Class: {class_label} (Confidence: {prediction[0][0]:.4f})")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(r"PycharmProjects/GPU_usage/myenv/model_weights_03/vgg_model_new_01.h5")
print("Model loaded successfully.")

In [ ]:
for layer in model.layers:
    if 'block5' in layer.name or 'block4' in layer.name: 
        layer.trainable = True


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=1e-5),
    metrics=['accuracy']
)


In [ ]:
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_finetune.history['accuracy'], label='Train Acc')
plt.plot(history_1.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_finetune.history['loss'], label='Train Loss')
plt.plot(history_finetune.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
filepath = r"C:\Users\joeca\PycharmProjects\GPU_usage\myenv\model_weights_03\vgg_model_new_02.h5"
model.save(filepath, overwrite=True)
print('model saved at ',filepath)

In [ ]:
val_accuracies = history_finetune.history['val_accuracy']
val_losses = history_finetune.history['val_loss']

avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)
avg_val_loss = sum(val_losses) / len(val_losses)

print(f"Average Validation Accuracy: {avg_val_accuracy * 100:.2f}%")
print(f"Average Validation Loss: {avg_val_loss:.4f}")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

test_generator.reset()

pred_probs = model.predict(test_generator, verbose=1)
pred_classes = (pred_probs > 0.5).astype(int).reshape(-1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

cm = confusion_matrix(true_classes, pred_classes)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print("\nClassification Report:")
print(classification_report(true_classes, pred_classes, target_names=class_labels))


In [5]:
from tensorflow.keras.models import load_model

model = load_model(r"PycharmProjects/GPU_usage/myenv/model_weights_03/vgg_model_new_02.h5")
print("Model loaded successfully.")

Model loaded successfully.


In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

img_path = r"C:\Users\joeca\Downloads\chest_xray\test\PNEUMONIA\person80_bacteria_392.jpeg"

img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0 

In [15]:
prediction = model.predict(img_array)

class_label = "PNEUMONIA" if prediction[0][0] > 0.5 else "NORMAL"

print(f"Predicted Class: {class_label} (Confidence: {prediction[0][0]:.4f})")

1/1 [==============================] - 0s 19ms/step
Predicted Class: PNEUMONIA (Confidence: 0.9945)
